In [1]:
import pandas as pd
import sklearn
import altair as alt

import re 
import string

import pandas as pd
import sklearn
import pickle
from wordcloud import WordCloud 

from nltk.corpus import stopwords #stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 
import spacy #used for lemmatization nltk.download('stopwords')
import gensim
from gensim.utils import simple_preprocess

from tqdm import tqdm



#import pyLDAvis.gensim_models
from gensim.models.coherencemodel import CoherenceModel 
#stop_words=set(nltk.corpus.stopwords.words('english'))

# Topic model
from bertopic import BERTopic

# Dimension reduction
from umap import UMAP

# Text preprocessiong
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()

# Remove stopwords
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andrewmartinson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andrewmartinson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/andrewmartinson/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andrewmartinson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#Loads the CFPB Complaints Database

df=pd.read_csv('complaints.csv', low_memory=False)
df['Date_received_dt'] = pd.to_datetime(df['Date received'], format='%Y/%m/%d')

In [3]:
#create a smaller dataframe based on dates being used for analysis
df_date = df[(df['Date received'] >= '2015-01-01') & (df['Date received'] <= '2016-12-31')]

In [4]:

#Identify the 5 banks or financial services companies with the most complaints
#Don't run if comparing the same banks over a period of time, only run for the first time period
top5=df_date.groupby('Company')['Complaint ID'].count().reset_index(name='count') \
            .sort_values(['count'], ascending=False) \
            .head(5)


#create list of top 5 companies for use in analysis
top_5_list = list(top5['Company'])


In [7]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing text = text.lower()'''
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[x*\/]', '', text) #remove those dates with the format of XX/XX OR XXXX/XX/XX
    text = re.sub('x', '', text)#remove \n
    text = re.sub('\n', '', text)#remove \n
    text = re.sub("[\(\[].*?[\)\]]", "", text) #remove bracketed text that appears like a list
    return text

round1 = lambda x: clean_text(x)



#Cycle through the bank data for comparison purposes
def extract_topics(list_of_companies, df_date):

    for company in tqdm(list_of_companies):
        df_bank = None
        data_clean = None
        topic_model = None
        topics = None
        probabilities = None
        topic_model = None


        df_bank = df_date[df_date['Company'] == company]

        #get rid of rows with null values in the customer narrative
        df_bank = df_bank.dropna(subset=['Consumer complaint narrative'])

        # Text cleaning - lowercase, remove punctuations and remove words with numbers.
        data_clean = pd.DataFrame(df_bank['Consumer complaint narrative'].apply(round1)) 

        # Remove stopwords
        data_clean['narrative_without_stopwords'] = data_clean['Consumer complaint narrative'].apply(lambda x: ' '.join([w for w in x.split() if w.lower() not in stopwords]))# Lemmatization
        data_clean['narrative_lemmatized'] = data_clean['narrative_without_stopwords'].apply(lambda x: ' '.join([wn.lemmatize(w) for w in x.split() if w not in stopwords]))# Take a look at the data

        #Must reset index to otherwise BERTopic will throw errors
        data_clean.reset_index(drop=True, inplace=True)

        # Initiate BERTopic
        # topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)
        topic_model = BERTopic(language="english", calculate_probabilities=True).fit(data_clean['narrative_without_stopwords'])

        #get the topics from the model
        tm_df = topic_model.get_topic_info()

        #save the topics into a csv file
        filenme = company+'_topics_2018-19'
        savepath = filenme+'.csv'
        tm_df.to_csv(path_or_buf=savepath)

        #save the model for later use
        topic_model.save(company+'_model')
    
    

In [ ]:
#create a smaller dataframe based on dates being used for analysis
df_date = None
df_date = df[(df['Date received'] >= '2018-01-01') & (df['Date received'] <= '2019-12-31')]
extract_topics(top_5_list, df_date)

  0%|                                                     | 0/5 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/andrewmartinson/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
 20%|████████▏                                | 1/5 [21:58<1:27:54, 1318.67s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/andrewmartinson/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
 40%|████████████████▍                        | 2/5 [40:38<1:00:05, 1201.79s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/andrewmartinson/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
 60%|█████████████████████████▊                 | 3/5 [59:36<39:05, 1172.67s/it]

In [12]:
# Initiate UMAP
umap_model = None
umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0,
                  metric='cosine', 
                  random_state=42)
